# Fine-Tuned Model Evaluation

In notebook 5, samples were generated for training and evaluating the Language Model (LLM) using Cross-Validation. Five folds were created for each type of ratio (Current, Quick, and Cash ratio), resulting in 15 training samples and 15 test samples. The base model was trained independently with each training sample, resulting in 15 trained models. In this notebook, the goal is to evaluate each of the trained models using its respective test sample. Therefore, steps 2, 3, 4 and 5 will be repeated for each trained model and its corresponding test file.

# 1. Initialize environment and model configuration

Import necessary libraries and modules from the torch and transformers packages. 

In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)

Set up configurations for quantization, offering choices for both 4-bit and 8-bit quantization.

In [2]:
use_4bit = True
use_8bit = True
quant_type_4bit = "nf4"
quant_type_8bit = "nf8"
use_nested_quant = True
compute_dtype_bfloat16 = torch.bfloat16
compute_dtype_float16 = torch.float16
device_mapping = {"": 0}

""" Alternative parameters:
load_in_8bit_fp32_cpu_offload=True,  # Enable CPU offloading for certain layers
bnb_8bit_blocksparse_layout=None,  # Block-sparse layout, use None for dense models
bnb_8bit_custom_kernel=False,  # Use custom kernel, false by default
bnb_8bit_cpu_offload=True,  # Enable CPU offloading
bnb_8bit_cpu_offload_dtype=torch.float32,  # Data type for CPU offloaded tensors
bnb_8bit_cpu_offload_use_pin_memory=True,  # Use pinned memory for CPU offloading
bnb_8bit_cpu_offload_use_fast_fp32_to_fp16_conversion=False  # Use fast conversion from FP32 to FP16
"""

bnb_config_4bit = BitsAndBytesConfig(
    load_in_4bit = use_4bit,
    bnb_4bit_quant_type = quant_type_4bit,
    bnb_4bit_use_double_quant = use_nested_quant,
    bnb_4bit_compute_dtype = compute_dtype_bfloat16
)
bnb_config_8bit = BitsAndBytesConfig(
    load_in_8bit = use_8bit,
    bnb_8bit_quant_type = quant_type_8bit,
    bnb_8bit_use_double_quant = use_nested_quant,
    bnb_8bit_compute_dtype = compute_dtype_float16
)

# 2. Load pretrained model

Define the path to the fine-tuned model directory.

In [3]:
model_path = "../Training/Fine-Tuned_models/cash_ratio_k5/model"

Load the tokenizer and model by selecting one of the configurations provided (4-bit or 8-bit).

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, quantization_config = bnb_config_4bit, device_map = device_mapping)

bin C:\ProgramData\anaconda3\envs\tfg\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

C:\ProgramData\anaconda3\envs\tfg\Lib\site-packages\transformers\generation\configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
C:\ProgramData\anaconda3\envs\tfg\Lib\site-packages\transformers\generation\configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


Print the loaded model for inspection or verification.

In [5]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


# 3. Load sample: formatting the DataFrame

Import the pandas library and set an option to display the maximum column width to None (unlimited width). Define the path, file name and file type (extension) for the CSV file to be read. Read the CSV file into a pandas DataFrame using the specified path and file details. Display the first few rows of the DataFrame.

In [6]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

path = '../Sample_generation/Cross_Validation_Sample/Cross_Validation_CSV_Files/'
file = 'test_set_Cash_5'
type_file = '.csv'

dataframe = pd.read_csv(path+file+type_file)

dataframe

,instruction,analysis,ratio,company
0,"Cash ratio of a company is 0.45 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has increased to 0.58, placing its value in the category defined as greater than 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","In 2021, the company had more than a reasonable amount of cash to cover a significant portion of its short-term liabilities. This suggests that the organization was likely able to meet its immediate financial obligations without significant challenges but a ratio over 0.40 reveals and excess of cash. The increase in the Cash ratio indicates that the company had even more cash relative to its short-term liabilities in 2022 compared to 2021. While having ample cash is generally positive for liquidity, it is important to consider that excessively high cash ratios might indicate that the company is not effectively utilizing its resources.",Cash,Carrier
1,"Cash ratio of a company is 0.38 in 2021, placing its value in the category defined as between 0.20 and 0.40. In 2022, Cash ratio has decreased to 0.32, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","The company maintained a relatively healthy Cash ratio in both years with a ratio between 0.20 and 0.40, indicating a reasonable ability to cover short-term obligations. The slight decrease in the Cash ratio in 2022 suggests a marginal reduction in the company's ability to cover short-term liabilities with available cash.",Cash,Costco
2,"Cash ratio of a company is 0.52 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.48, placing its value in the category defined as greater than 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","Cash ratios for 2021 and 2022 are upper 0.40, so the company had more than a reasonable amount of cash to cover a significant portion of its short-term liabilities, which indicates an excess of liquidity. In 2022, Cash ratio decreased but still upper 0.40. And while having ample cash is generally positive for liquidity, excessively high Cash ratios might indicate that the company is not effectively utilizing its resources.",Cash,Fedex
3,"Cash ratio of a company is 0.47 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.36, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","Cash ratio in 2021 suggests that the company had a higher level of cash relative to its short-term liabilities. This may indicate an inefficient use of cash, as excessively high cash ratios might imply that the company is not optimizing its resources. In 2022, the subsequent decrease and fall of the Cash ratio into the range between 0.20 and 0.40 signifies a potential adjustment in the company's approach to cash management, successfully mitigating the excess of cash experienced in the previous year.",Cash,Hilton
4,"Cash ratio of a company is 0.20 in 2021, placing its value in the category defined as between 0.20 and 0.40. In 2022, Cash ratio has increased to 0.25, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","In 2021, the Cash ratio falls into the range between 0.20 and 0.40, which means that the company has a reasonable amount of cash on hand to cover a significant portion of its short-term liabilities. It suggests that the organization is likely to meet its immediate financial obligations without significant challenges. In 2022, the Cash ratio value is still within the same range, indicating that the company continues to maintain a reasonable amount of cash. The slight increase from 2021 to 2022 suggests a potential improvement in liquidity, as the company has a slightl

Define a function named 'format_df' that takes the DataFrame and file name as input and performs the following operations: Reindex the DataFrame with columns 'company', 'ratio', 'instruction', and 'analysis'. Rename the 'analysis' column to 'analysis_expected'. Add a new column 'analysis_generated' with all values set to None. Add another column 'iteration' that contains the k-fold number. Return the DataFrame.

In [7]:
def format_df(df,file_name):
    new_df = df.reindex(['company', 'ratio', 'instruction', 'analysis'], axis=1)
    new_df = new_df.rename(columns={'analysis': 'analysis_expected'})
    new_df['analysis_generated'] = None
    new_df['iteration'] = int(file_name.split('_')[-1])
    
    return new_df

Call the function and display the first few rows of the resulting DataFrame.

In [8]:
df_formatted = format_df(dataframe, file)

In [9]:
df_formatted

,company,ratio,instruction,analysis_expected,analysis_generated,iteration
0,Carrier,Cash,"Cash ratio of a company is 0.45 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has increased to 0.58, placing its value in the category defined as greater than 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","In 2021, the company had more than a reasonable amount of cash to cover a significant portion of its short-term liabilities. This suggests that the organization was likely able to meet its immediate financial obligations without significant challenges but a ratio over 0.40 reveals and excess of cash. The increase in the Cash ratio indicates that the company had even more cash relative to its short-term liabilities in 2022 compared to 2021. While having ample cash is generally positive for liquidity, it is important to consider that excessively high cash ratios might indicate that the company is not effectively utilizing its resources.",None,5
1,Costco,Cash,"Cash ratio of a company is 0.38 in 2021, placing its value in the category defined as between 0.20 and 0.40. In 2022, Cash ratio has decreased to 0.32, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","The company maintained a relatively healthy Cash ratio in both years with a ratio between 0.20 and 0.40, indicating a reasonable ability to cover short-term obligations. The slight decrease in the Cash ratio in 2022 suggests a marginal reduction in the company's ability to cover short-term liabilities with available cash.",None,5
2,Fedex,Cash,"Cash ratio of a company is 0.52 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.48, placing its value in the category defined as greater than 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","Cash ratios for 2021 and 2022 are upper 0.40, so the company had more than a reasonable amount of cash to cover a significant portion of its short-term liabilities, which indicates an excess of liquidity. In 2022, Cash ratio decreased but still upper 0.40. And while having ample cash is generally positive for liquidity, excessively high Cash ratios might indicate that the company is not effectively utilizing its resources.",None,5
3,Hilton,Cash,"Cash ratio of a company is 0.47 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.36, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","Cash ratio in 2021 suggests that the company had a higher level of cash relative to its short-term liabilities. This may indicate an inefficient use of cash, as excessively high cash ratios might imply that the company is not optimizing its resources. In 2022, the subsequent decrease and fall of the Cash ratio into the range between 0.20 and 0.40 signifies a potential adjustment in the company's approach to cash management, successfully mitigating the excess of cash experienced in the previous year.",None,5
4,IBM,Cash,"Cash ratio of a company is 0.20 in 2021, placing its value in the category defined as between 0.20 and 0.40. In 2022, Cash ratio has increased to 0.25, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","In 2021, the Cash ratio falls into the range between 0.20 and 0.40, which means that the company has a reasonable amount of cash on hand to cover a significant portion of its short-term liabilities. It suggests that the organization is likely to meet its immediate financial obligations without significant challenges. In 2022, the Cash ratio value is still within the same range, indicating that the company continues to maintain a reasonable amount of cash. The slight increase from 2021 to 2022 

# 4. Generate expected analysis

Import necessary modules from PyTorch for working with datasets and dataloaders. Define a custom dataset class named CustomDataset that inherits from Dataset. This class takes a DataFrame as input during initialization and implements methods for length (__len__) and getting individual items (__getitem__). It extracts the 'instruction' column from the DataFrame for each item.

Define a function generate_analysis that takes a DataFrame (dataframe) and a text generation pipeline (text_generation_pipeline) as input. Inside the function: Create a CustomDataset using the DataFrame passed as input of the function. Use a DataLoader to load the dataset with a batch size of 1. Iterate over the DataLoader batches, extracting the 'question' and generating a corresponding ratio analysis using the LLM. Store the generated answer in the 'analysis_generated' column of the DataFrame.

In [10]:
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        question = self.dataframe.iloc[idx]['instruction']
        return {
            'question': question
        }

def generate_analysis(dataframe, text_generation_pipeline):
    dataset = CustomDataset(dataframe)
    
    dataloader = DataLoader(dataset, batch_size=1)

    for idx, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
        question = batch['question'][0]
        
        prompt = f"Question: {question}\nAnswer:"
        
        sequences = text_generation_pipeline(
            prompt,
            max_length=512,
            do_sample=True,
            top_k=10,
            temperature = 0.6,
            return_full_text=False,
        )
        
        generated_text = sequences[0]['generated_text']
        dataframe.at[idx, 'analysis_generated'] = generated_text

    return dataframe

Set up a text generation pipeline using the LLM model and tokenizer.

In [11]:
text_generation_pipeline = pipeline('text-generation', model = model, tokenizer = tokenizer)

Call the generate_analysis function using the DataFrame created in step 3 and the text generation pipeline for updating the DataFrame with the analysis generated by the LLM. Display resulting DataFrame.

In [12]:
df_analyzed = generate_analysis(df_formatted, text_generation_pipeline)

  0%|          | 0/10 [00:00<?, ?it/s]

In [13]:
df_analyzed

,company,ratio,instruction,analysis_expected,analysis_generated,iteration
0,Carrier,Cash,"Cash ratio of a company is 0.45 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has increased to 0.58, placing its value in the category defined as greater than 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","In 2021, the company had more than a reasonable amount of cash to cover a significant portion of its short-term liabilities. This suggests that the organization was likely able to meet its immediate financial obligations without significant challenges but a ratio over 0.40 reveals and excess of cash. The increase in the Cash ratio indicates that the company had even more cash relative to its short-term liabilities in 2022 compared to 2021. While having ample cash is generally positive for liquidity, it is important to consider that excessively high cash ratios might indicate that the company is not effectively utilizing its resources.","In 2021, the company had a high Cash ratio, indicating an excess of liquidity. In 2022, the company continued to maintain a high Cash ratio, suggesting that the company continues to carry a significant amount of cash.",5
1,Costco,Cash,"Cash ratio of a company is 0.38 in 2021, placing its value in the category defined as between 0.20 and 0.40. In 2022, Cash ratio has decreased to 0.32, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","The company maintained a relatively healthy Cash ratio in both years with a ratio between 0.20 and 0.40, indicating a reasonable ability to cover short-term obligations. The slight decrease in the Cash ratio in 2022 suggests a marginal reduction in the company's ability to cover short-term liabilities with available cash.","The company's Cash ratio decreased from 0.38 in 2021 to 0.32 in 2022, indicating a reduction in its liquidity. Both ratios fall within the range of 0.20 to 0.40, suggesting that the company maintains an adequate level of cash to cover a significant portion of its short-term liabilities.",5
2,Fedex,Cash,"Cash ratio of a company is 0.52 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.48, placing its value in the category defined as greater than 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","Cash ratios for 2021 and 2022 are upper 0.40, so the company had more than a reasonable amount of cash to cover a significant portion of its short-term liabilities, which indicates an excess of liquidity. In 2022, Cash ratio decreased but still upper 0.40. And while having ample cash is generally positive for liquidity, excessively high Cash ratios might indicate that the company is not effectively utilizing its resources.","In 2021, the company had a high Cash ratio, indicating an excess of liquidity. In 2022, the Cash ratio has decreased, but it is still above 0.40, suggesting that the company continues to possess an excess of liquidity.",5
3,Hilton,Cash,"Cash ratio of a company is 0.47 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.36, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","Cash ratio in 2021 suggests that the company had a higher level of cash relative to its short-term liabilities. This may indicate an inefficient use of cash, as excessively high cash ratios might imply that the company is not optimizing its resources. In 2022, the subsequent decrease and fall of the Cash ratio into the range between 0.20 and 0.40 signifies a potential adjustment in the company's approach to cash management, successfully mitigating the excess of cash experienced in the previous year.","In 2021, the company had a high Cash ratio, indicating an excess of liquidity. In 2022, t

# 5. Sentence similarity with Sentence Transformers

Import necessary libraries such as SentenceTransformer, util from sentence_transformers, pandas and torch. Define a function named calculate_cosine_similarity that takes a DataFrame (df) and a Sentence Transformer model (model) as input: Iterate through rows of the DataFrame, calculating cosine similarity between the embeddings of "analysis_expected" and "analysis_generated" columns using the Sentence Transformer model. 
Append the cosine similarity values to a list and add a new column 'cosine_similarity' to the DataFrame. 
Return the modified DataFrame.

In [14]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch

pd.set_option('display.max_colwidth', None)

def calculate_cosine_similarity(df, model):
    cosine_similarities = []
    
    for index, row in df.iterrows():
        expected_embedding = model.encode([row['analysis_expected']])
        generated_embedding = model.encode([row['analysis_generated']])
        cosine_similarity_matrix = util.cos_sim(torch.tensor(expected_embedding), torch.tensor(generated_embedding)).numpy()
        cosine_similarity = cosine_similarity_matrix[0][0]
        cosine_similarities.append(cosine_similarity)

    df['cosine_similarity'] = cosine_similarities

    return df

Load Sentence Transformer Model: Specify the Sentence Transformer model name (st_model_name) as 'sentence-transformers/all-mpnet-base-v2'. Create an instance of the Sentence Transformer model (st_model) using the specified model name.

In [15]:
st_model_name = 'sentence-transformers/all-mpnet-base-v2'
st_model = SentenceTransformer(st_model_name)

Perform cosine similarity calculation using the calculate_cosine_similarity function and the Sentence Transformer model. Save the resulting DataFrame with added 'cosine_similarity' column to new CSV file.

In [16]:
df_similarity = calculate_cosine_similarity(df_analyzed, st_model)

In [17]:
df_similarity

,company,ratio,instruction,analysis_expected,analysis_generated,iteration,cosine_similarity
0,Carrier,Cash,"Cash ratio of a company is 0.45 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has increased to 0.58, placing its value in the category defined as greater than 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","In 2021, the company had more than a reasonable amount of cash to cover a significant portion of its short-term liabilities. This suggests that the organization was likely able to meet its immediate financial obligations without significant challenges but a ratio over 0.40 reveals and excess of cash. The increase in the Cash ratio indicates that the company had even more cash relative to its short-term liabilities in 2022 compared to 2021. While having ample cash is generally positive for liquidity, it is important to consider that excessively high cash ratios might indicate that the company is not effectively utilizing its resources.","In 2021, the company had a high Cash ratio, indicating an excess of liquidity. In 2022, the company continued to maintain a high Cash ratio, suggesting that the company continues to carry a significant amount of cash.",5,0.846915
1,Costco,Cash,"Cash ratio of a company is 0.38 in 2021, placing its value in the category defined as between 0.20 and 0.40. In 2022, Cash ratio has decreased to 0.32, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","The company maintained a relatively healthy Cash ratio in both years with a ratio between 0.20 and 0.40, indicating a reasonable ability to cover short-term obligations. The slight decrease in the Cash ratio in 2022 suggests a marginal reduction in the company's ability to cover short-term liabilities with available cash.","The company's Cash ratio decreased from 0.38 in 2021 to 0.32 in 2022, indicating a reduction in its liquidity. Both ratios fall within the range of 0.20 to 0.40, suggesting that the company maintains an adequate level of cash to cover a significant portion of its short-term liabilities.",5,0.831891
2,Fedex,Cash,"Cash ratio of a company is 0.52 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.48, placing its value in the category defined as greater than 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","Cash ratios for 2021 and 2022 are upper 0.40, so the company had more than a reasonable amount of cash to cover a significant portion of its short-term liabilities, which indicates an excess of liquidity. In 2022, Cash ratio decreased but still upper 0.40. And while having ample cash is generally positive for liquidity, excessively high Cash ratios might indicate that the company is not effectively utilizing its resources.","In 2021, the company had a high Cash ratio, indicating an excess of liquidity. In 2022, the Cash ratio has decreased, but it is still above 0.40, suggesting that the company continues to possess an excess of liquidity.",5,0.937251
3,Hilton,Cash,"Cash ratio of a company is 0.47 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.36, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","Cash ratio in 2021 suggests that the company had a higher level of cash relative to its short-term liabilities. This may indicate an inefficient use of cash, as excessively high cash ratios might imply that the company is not optimizing its resources. In 2022, the subsequent decrease and fall of the Cash ratio into the range between 0.20 and 0.40 signifies a potential adjustment in the company's approach to cash management, successfully mitigating the excess of cash experienced in the previous year.","In 2021, the company had a high Cash ratio, 

In [18]:
path = './Fine_Tuned_Evaluation/'
file = 'fine_tuned_cash_5_similarity'
type_file = '.csv'

df_similarity.to_csv(path+file+type_file, index=True)

# 6. Evaluation results

## 6.1 Create a DataFrame template

Define the 'index' and 'columns' for a DataFrame with specific labels for rows and columns. Create an empty DataFrame using the specified index and columns. This dataframe will store the evaluation results for each iteration and ratio type.

In [1]:
import pandas as pd

index = ['fine_tuned']
columns = ['current_ratio', 'quick_ratio', 'cash_ratio']

df_template = pd.DataFrame(index=index, columns=columns)

df_template

,current_ratio,quick_ratio,cash_ratio
fine_tuned,NaN,NaN,NaN


## 6.2 Calculate Cosine Similarity mean

Define a function named 'sentence_similarity_mean' that calculates the mean of the 'cosine_similarity' column in a given DataFrame and formats it with six decimal places. Define a function named 'store_evaluation' that updates the 'df_template' DataFrame with the mean cosine similarity for a specific ratio type using a given DataFrame.

In [2]:
def sentence_similarity_mean(df):
    mean = df['cosine_similarity'].sum() / len(df)
    formatted_mean = '{:.6f}'.format(mean)
    return formatted_mean
    
def store_evaluation(index, ratio_type, df_template, df_results):
    df_template.at[index, ratio_type] = sentence_similarity_mean(df_results)

## Current ratio

Retrieve the files containing cosine similarity results for Current ratio: set the path, file type and create an empty list 'dfs_to_concat'. Iterate through a range from 1 to 5, construct the file name, read the corresponding CSV file and append the DataFrame to 'df_results'. Finally, concatenate all the DataFrames into a single DataFrame that contains all the iterations. Display the resulting DataFrame and save it to a CSV file.

In [3]:
path = './Fine_Tuned_Evaluation/'
type_file = '.csv'
dfs_to_concat = []
for i in range (1,6):
    file = f'fine_tuned_current_{i}_similarity'
    dfs_to_concat.append(pd.read_csv(path + file + type_file, index_col=0))
df_results = pd.concat(dfs_to_concat, ignore_index=True)

In [4]:
df_results

,company,ratio,instruction,analysis_expected,analysis_generated,iteration,cosine_similarity
0,Ebay,Current,"Current ratio of a company is 1.97 in 2021, pl...","In 2021, the company had a relatively strong l...",The company's Current ratio increased from 1....,1,0.880280
1,Hasbro,Current,"Current ratio of a company is 1.52 in 2021, pl...","In 2021, company has a comfortable cushion to ...",The company's Current ratio decreased from 1....,1,0.886567
2,HP,Current,"Current ratio of a company is 0.91 in 2021, pl...",A Current ratio below 1.00 in both years indic...,The company's Current ratio decreased from 20...,1,0.944138
3,Marriott,Current,"Current ratio of a company is 0.57 in 2021, pl...",The company's Current ratio declined from 2021...,The company's Current ratio decreased from 0....,1,0.937046
4,Microsoft,Current,"Current ratio of a company is 2.08 in 2021, pl...","In 2021, Current ratio is over 2.00, indicatin...","In 2021, the company had a Current ratio grea...",1,0.928579
5,Netflix,Current,"Current ratio of a company is 0.95 in 2021, pl...","In 2021, the company's Current ratio of 0.95 i...","In 2021, the company's current assets are not...",1,0.776129
6,NVIDIA,Current,"Current ratio of a company is 4.09 in 2021, pl...",The company's Current ratio increased from 4.0...,The company's Current ratio in 2021 is signif...,1,0.946230
7,ProcterGamble,Current,"Current ratio of a company is 0.70 in 2021, pl...",The company's Current ratio declined from 0.70...,The company's Current ratio decreased from 20...,1,0.963211
8,Seagate,Current,"Current ratio of a company is 1.29 in 2021, pl...",The company's Current ratio decreased from 1.2...,The company's Current ratio decreased from 1....,1,0.991633
9,Walmart,Current,"Current ratio of a company is 0.97 in 2021, pl...",The company's Current ratio declined from 0.97...,The company's Current ratio decreased from 20...,1,0.967111


In [6]:
path = './Fine_Tuned_Evaluation/'
file = 'fine_tuned_current_k1_k5_similarity'
type_file = '.csv'
df_results.to_csv(path+file+type_file, index=True)

Set the 'ratio_name' to 'current_ratio'. Call 'store_evaluation' to update the 'df_template' DataFrame with mean cosine similarity values for the 'current_ratio'.

In [8]:
ratio_name = 'current_ratio'

store_evaluation(index[0], ratio_name, df_template, df_results)

In [9]:
df_template

,current_ratio,quick_ratio,cash_ratio
fine_tuned,0.896326,NaN,NaN


## Quick ratio

Retrieve the files containing cosine similarity results for Quick ratio: set the path, file type and create an empty list 'dfs_to_concat'. Iterate through a range from 1 to 5, construct the file name, read the corresponding CSV file and append the DataFrame to 'df_results'. Finally, concatenate all the DataFrames into a single DataFrame that contains all the iterations. Display the resulting DataFrame and save it to a CSV file.

In [10]:
path = './Fine_Tuned_Evaluation/'
type_file = '.csv'
dfs_to_concat = []
for i in range (1,6):
    file = f'fine_tuned_quick_{i}_similarity'
    dfs_to_concat.append(pd.read_csv(path + file + type_file, index_col=0))
df_results = pd.concat(dfs_to_concat, ignore_index=True)

In [11]:
df_results

,company,ratio,instruction,analysis_expected,analysis_generated,iteration,cosine_similarity
0,Ebay,Quick,"Quick ratio of a company is 1.73 in 2021, plac...","In 2021, the company maintained a Quick ratio ...",The company's Quick ratio in 2021 and 2022 in...,1,0.900173
1,Hasbro,Quick,"Quick ratio of a company is 1.03 in 2021, plac...",The company exhibited a favorable Quick ratio ...,"In 2021, the Quick ratio ranged between 1.00 ...",1,0.942912
2,HP,Quick,"Quick ratio of a company is 0.69 in 2021, plac...",The company's Quick ratio in both years indica...,The company's Quick ratio declined from 0.69 ...,1,0.902829
3,Marriott,Quick,"Quick ratio of a company is 0.53 in 2021, plac...",Experiencing a decline from the preceding year...,The company's Quick ratio declined from 0.53 ...,1,0.939371
4,Microsoft,Quick,"Quick ratio of a company is 2.05 in 2021, plac...",The company's Quick ratio has experienced a de...,The company's Quick ratio decreased from 2.05...,1,0.937334
5,Netflix,Quick,"Quick ratio of a company is 0.95 in 2021, plac...",The company's Quick ratio increased from 0.95 ...,"In 2021, the Quick ratio suggests a difficult...",1,0.847920
6,NVIDIA,Quick,"Quick ratio of a company is 3.56 in 2021, plac...",The company's Quick ratio increased from 3.56 ...,The company's Quick ratio increased from 3.56...,1,0.921628
7,ProcterGamble,Quick,"Quick ratio of a company is 0.45 in 2021, plac...",The company's Quick ratio declined from 0.45 i...,The company's Quick ratio declined from 0.45 ...,1,0.975732
8,Seagate,Quick,"Quick ratio of a company is 0.88 in 2021, plac...",The company's Quick ratio declined from 0.88 i...,The company's Quick ratio declined from 0.88 ...,1,0.972494
9,Walmart,Quick,"Quick ratio of a company is 0.26 in 2021, plac...",The company's Quick ratio remained at the valu...,The company's Quick ratio in 2021 and 2022 in...,1,0.952234


In [12]:
path = './Fine_Tuned_Evaluation/'
file = 'fine_tuned_quick_k1_k5_similarity'
type_file = '.csv'
df_results.to_csv(path+file+type_file, index=True)

Set the 'ratio_name' to 'quick_ratio'. Call 'store_evaluation' to update the 'df_template' DataFrame with mean cosine similarity values for the 'quick_ratio'.

In [13]:
ratio_name = 'quick_ratio'
store_evaluation(index[0], ratio_name, df_template, df_results)

## Cash ratio

Retrieve the files containing cosine similarity results for Cash ratio: set the path, file type and create an empty list 'dfs_to_concat'. Iterate through a range from 1 to 5, construct the file name, read the corresponding CSV file and append the DataFrame to 'df_results'. Finally, concatenate all the DataFrames into a single DataFrame that contains all the iterations. Display the resulting DataFrame and save it to a CSV file.

In [14]:
path = './Fine_Tuned_Evaluation/'
type_file = '.csv'
dfs_to_concat = []
for i in range (1,6):
    file = f'fine_tuned_cash_{i}_similarity'
    dfs_to_concat.append(pd.read_csv(path + file + type_file, index_col=0))
df_results = pd.concat(dfs_to_concat, ignore_index=True)

In [15]:
df_results

,company,ratio,instruction,analysis_expected,analysis_generated,iteration,cosine_similarity
0,Ebay,Cash,"Cash ratio of a company is 0.30 in 2021, placi...",The company maintained a reasonable level of l...,"In 2021, the company had a reasonable amount ...",1,0.891856
1,Hasbro,Cash,"Cash ratio of a company is 0.42 in 2021, placi...","Cash ratio in 2021 is above 0.40, indicating t...","In 2021, the company had an excess of liquidi...",1,0.852403
2,HP,Cash,"Cash ratio of a company is 0.19 in 2021, placi...","The company's Cash ratios in both years, 2021 ...",The company's Cash ratio has decreased from 0...,1,0.920409
3,Marriott,Cash,"Cash ratio of a company is 0.22 in 2021, placi...",The company witnessed a substantial reduction ...,"In 2021, the company had a reasonable amount ...",1,0.895307
4,Microsoft,Cash,"Cash ratio of a company is 0.16 in 2021, placi...","The company's Cash ratios in both years, 2021 ...",The company's Cash ratio has decreased from 0...,1,0.913789
5,Netflix,Cash,"Cash ratio of a company is 0.71 in 2021, placi...",The company's declining Cash ratio from 0.71 i...,The company's Cash ratio decreased from 0.71 ...,1,0.944225
6,NVIDIA,Cash,"Cash ratio of a company is 0.22 in 2021, placi...",The company experienced a increase in its Cash...,"In 2021, the company had a reasonable amount ...",1,0.866806
7,ProcterGamble,Cash,"Cash ratio of a company is 0.31 in 2021, placi...",The company's Cash ratio decreased from 0.31 i...,The company's Cash ratio decreased from 0.31 ...,1,0.976710
8,Seagate,Cash,"Cash ratio of a company is 0.41 in 2021, placi...",The company's Cash ratio decreased from 0.41 i...,"In 2021, the company had an excess of liquidi...",1,0.948925
9,Walmart,Cash,"Cash ratio of a company is 0.19 in 2021, placi...",The company's Cash ratio decreased from 0.19 i...,The company's Cash ratio has decreased from 0...,1,0.964634


In [16]:
path = './Fine_Tuned_Evaluation/'
file = 'fine_tuned_cash_k1_k5_similarity'
type_file = '.csv'
df_results.to_csv(path+file+type_file, index=True)

Set the 'ratio_name' to 'cash_ratio'. Call 'store_evaluation' to update the 'df_template' DataFrame with mean cosine similarity values for the 'cash_ratio'.

In [17]:
ratio_name = 'cash_ratio'

store_evaluation(index[0], ratio_name, df_template, df_results)

## 6.3 Evaluation summary

Display the updated DataFrame template with cosine similarity means and save it to a CSV file.

In [18]:
df_template

,current_ratio,quick_ratio,cash_ratio
fine_tuned,0.896326,0.898498,0.893478


In [19]:
path = './Fine_Tuned_Evaluation/'
file = 'fine_tuned_similarity_summary'
type_file = '.csv'

df_template.to_csv(path+file+type_file, index=True)